<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Optimizing with Decision Frames  

Work in Progress 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.4"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.4"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.4 Microsoft.Extensions.Configuration.Json, 9.0.4

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.4"
#r "nuget: Microsoft.SemanticKernel, 1.47"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.4 Microsoft.SemanticKernel, 1.47.0

Using Azure OpenAI Service


### Step 2 - Decision Frames (Thinking Hats) Scenario for a Major Purchase 

> 📜 "The mere formulation of a problem is far more essential than its solution, which may be merely a matter of mathematical or experimental skill. To raise new questions, new possibilities, to regard old problems from a new angle requires creative imagination and marks real advances in science."
>
> -- <cite>Albert Einstein (Theoretical Physicist, Best known for developing the theory of relativity)</cite>  

The code below will list the potential decision frames (using thinking hats frames) that may be appropriate to form different approaches for purchasing a house. Generative AI can be used to easily "put on a hat" (approach a problem with a specific decision frame) much easier than a human decision maker.  

In [3]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions: 
List an approach for each of the six hats. 
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Edward de Bono's Six Thinking Hats is a structured decision-making tool that looks at a problem from six distinct perspectives. Here’s how you could apply each hat to purchasing a new home:

---

### 1. **White Hat (Facts and Information)**  
Focus: What information do you have? What more do you need to know?  

**Approach:**  
- Gather essential data: your budget, income, mortgage pre-approval, credit score, etc.  
- Research market trends: What are the housing prices in your desired area? Are prices rising or falling?  
- Create a checklist of "must-have" home features (e.g., number of bedrooms, location, proximity to schools or work).  
- Investigate property taxes, HOA fees, utility expenses, and other ongoing costs.  
- Consult professionals (a realtor, mortgage lender, financial advisor).  
- Research neighborhoods: crime rates, schools, commute times, amenities, future development.  

---

### 2. **Red Hat (Emotions and Intuition)**  
Focus: What are your feelings and instincts 

This can be enhanced further by having GenAI suggest which of the decision frames might be the most useful for a high-stakes decision. Therefore, the prompt above can be optimized further by listing all the decision frames and then priorotizing which ones will have the most impact and influence on the house purchase decision.  

This allows the Artificial Intelligence automation to focus on the optimization of the decision process and the human on the actual decision selection and execution!

In [4]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesListPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions:
List an approach for each of the six hats in the order of impact & influence from most to least. 
From the priorotized list suggest which thinking hats (decision frames) to focus on. 
It can be one of the hats or any combination of the thinking hats. 
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesListPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Edward de Bono's Six Thinking Hats framework is a highly effective approach for structured decision-making. Below is a breakdown of how to apply each hat to purchasing a new home, prioritized based on impact and influence for this scenario.

---

### **Prioritized List of Hats Approach**

#### **1. White Hat: Information and Facts (High Priority)**  
**Approach:**  
- Gather all relevant facts and data about the properties you're considering: location, price, size, amenities, neighborhood, schools, commute distance, and market trends.  
- Research mortgage rates, financing options, property taxes, maintenance costs, and utilities.  
- Assess the reliability of your data sources and identify gaps in the information that need to be filled.

**Reason for Priority:**  
Without accurate and thorough information, other considerations may be misguided or incomplete. Decisions based on incorrect or incomplete data can lead to costly mistakes.  

---

#### **2. Yellow Hat: Benefits and Optimism

So far, (given the house purchase decision) Generative AI has helped create decision frames with the six thinking hats framework, rank them from most impact & influence from most to least and select the top 3 decision frames. The next logical step is to leverage Generative AI to apply the selected 3 decision frames for the house purchase decision.  

This can be done in two different ways:  
1) **Single Decision Prompt:** Instruct (prompt) the GenAI model "think internally" on the decision frames, ranking, selecting the top decision frames and then finally output the applied decision frames for the house purchase decision.
2) **Workflow Decision Prompts:** This approach uses multiple prompts together to arrive at a decision approach. This breaks the problem down into smaller components and executes each component in sequence. The value is that each step can be optimized individually and each intermediate output is clearly available for analysis. This is ideal for sufficiently important decision problems.  

#### Single Decision Prompt 

In [5]:
// Set the overall system prompt to behave like a Decision Intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesListPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions:

<Do Not Show these steps>
INTERNALLY THINK about an approach for each of the six hats in the order of impact & influence from most to least. 
From the priorotized list INTERNALLY PROCESS which thinking hats (decision frames) to focus on. 
It can be one of the hats or any combination of the thinking hats. 
Let's call that internal list "selected decision frames"
</Do Not Show these steps>

Only output these steps to the user: 
Finally, from the internal "selected decision frames", create an approach for each for purchasing a house.  
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesListPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Here’s a structured approach for purchasing a house based on the "selected decision frames" using Edward de Bono's Six Thinking Hats:

---

### Practical Thinking (Black Hat & White Hat Combination: Critical + Objective)
Focus on risks, facts, and logical considerations:
1. **Assess Financial Viability**: Review your budget, credit score, down payment capacity, and mortgage approval potential. Ensure you're comfortable with long-term affordability.
2. **Market Analysis**: Research property values, price trends, and the overall stability of the real estate market in your desired area.
3. **Inspection and Risks**: Consider the condition of the house (e.g., structural integrity, age, needed repairs) and assess potential risks such as future maintenance, location hazards (e.g., flood zones), or declining neighborhood trends.

---

### Emotional Thinking (Red Hat)
Tap into your intuition and emotional response:
1. **Gauge Emotional Connection**: Reflect on how you and your family feel about